In [ ]:
# Uninstall potentially conflicting packages
!pip uninstall -y transformers accelerate unsloth torch torchvision torchaudio

# Install base packages
!pip install unsloth

# Install dependencies
!pip install -q transformers accelerate peft
!pip install -q datasets evaluate bitsandbytes trl
!pip install -q torch torchvision torchaudio

# Install Colab-optimized unsloth
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install other tools
!pip install pandas scikit-learn
!pip install -q ipywidgets

In [ ]:
import os
import pandas as pd
import torch
import gc
from datasets import Dataset
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
from peft import PeftModel
import warnings
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

# Data and model paths
data_dir = "/content/drive/MyDrive/ai_dataset"
save_dir = "/content/drive/MyDrive/ai_detection_model"
final_model_path = os.path.join(save_dir, "final_model")

# Fix random seeds for reproducibility
import random
import numpy as np

def set_seeds(seed=3407):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seeds()

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

clear_memory()
torch.set_float32_matmul_precision('high')

# ===================== Dataset Preparation =====================
human_xlsx_path = os.path.join(data_dir, "human.xlsx")
ai_xlsx_path = os.path.join(data_dir, "ai.xlsx")

df_human = pd.read_excel(human_xlsx_path)
df_human = df_human[['abstract']]
df_human['is_ai_generated'] = 'False'

df_ai = pd.read_excel(ai_xlsx_path)
df_ai = df_ai[['abstract']]
df_ai['is_ai_generated'] = 'True'

df = pd.concat([df_human, df_ai], ignore_index=True)
dataset = Dataset.from_pandas(df)

# Limit the dataset sample size, e.g., using 1000 samples for testing
max_samples = 1000
if max_samples > 0 and max_samples < len(dataset):
    dataset = dataset.shuffle(seed=3407).select(range(min(max_samples, len(dataset))))

train_val_idx, test_idx = train_test_split(
    range(len(dataset)),
    test_size=0.2,
    random_state=3407
)

train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.125, # 0.125 * 0.8 = 0.1 (final 70%:10%:20%)
    random_state=3407
)

def process_training_example(example):
    text = example['abstract']
    is_ai_generated = example['is_ai_generated']
    prompt = (
        "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
        f"Given Text: {text}\n\n"
        "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        f"Answer: {str(is_ai_generated)}"
    )
    return {"text": prompt}

def process_test_example(example):
    text = example['abstract']
    prompt = (
        "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
        f"Given Text: {text}\n\n"
        "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        "Answer:"
    )
    return prompt

train_examples = [process_training_example(dataset[i]) for i in train_idx]
eval_examples = [process_training_example(dataset[i]) for i in val_idx]
test_examples = [dataset[i] for i in test_idx]

train_dataset = Dataset.from_list(train_examples)
eval_dataset = Dataset.from_list(eval_examples)

# Keep true labels in test_dataset for evaluating predictions
test_dataset = Dataset.from_list([
    {'abstract': item['abstract'], 'is_ai_generated': item['is_ai_generated']} for item in test_examples
])

print(f"Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}, Test size: {len(test_dataset)}")

# ===================== Load Final Model =====================
base_model_name = "unsloth/Meta-Llama-3.1-8B"  # Same base model as during training

clear_memory()

base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=2048,
    load_in_4bit=True,
)

model = PeftModel.from_pretrained(base_model, final_model_path)
model.eval()

# Enter inference mode
FastLanguageModel.for_inference(model)

def parse_prediction(response: str) -> bool:
    response = response.lower().strip()
    if "true" in response and "false" not in response:
        return True   # Predicted as AI generated
    elif "false" in response and "true" not in response:
        return False  # Predicted as Human generated
    else:
        return False

# ===================== Inference and Evaluation on Test Set =====================
batch_size = 16
test_examples_list = list(test_dataset)
total_batches = (len(test_examples_list) + batch_size - 1) // batch_size
all_predictions = []

with torch.no_grad():
    for i in range(0, len(test_examples_list), batch_size):
        if i % (batch_size * 10) == 0:
            print(f"Processing batch {i//batch_size}/{total_batches}")

        batch = test_examples_list[i:i + batch_size]
        prompts = [process_test_example(example) for example in batch]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=8,
            temperature=0.1,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

        input_length = inputs['input_ids'].shape[1]
        responses = tokenizer.batch_decode(
            [output[input_length:] for output in outputs],
            skip_special_tokens=True
        )

        batch_predictions = [parse_prediction(response) for response in responses]
        all_predictions.extend(batch_predictions)

print(f"Total predictions: {len(all_predictions)}")

# Convert true labels to boolean, True for AI generated, False for Human
true_labels = [row['is_ai_generated'] == 'True' for row in test_dataset]

# Save predictions and true labels to CSV, include correctness column
submission_df = pd.DataFrame({
    'ID': range(len(all_predictions)),
    'is_ai_generated_pred': all_predictions,
    'is_ai_generated_true': true_labels
})

submission_df['correctness'] = (submission_df['is_ai_generated_pred'] == submission_df['is_ai_generated_true'])

submission_path = os.path.join(save_dir, 'submission_with_truth.csv')
submission_df.to_csv(submission_path, index=False)
print(f"Submission with truth saved to {submission_path}")

print(submission_df.head())

# Output classification report
print("Classification Report:")
print(classification_report(true_labels, all_predictions, target_names=["Human","AI"]))

In [ ]:
max_samples = 5000

human_xlsx_path = os.path.join(data_dir, "human.xlsx")
ai_xlsx_path = os.path.join(data_dir, "ai.xlsx")

df_human = pd.read_excel(human_xlsx_path)
df_human = df_human[['abstract']]
df_human['is_ai_generated'] = 'False'

df_ai = pd.read_excel(ai_xlsx_path)
df_ai = df_ai[['abstract']]
df_ai['is_ai_generated'] = 'True'

df = pd.concat([df_human, df_ai], ignore_index=True)
dataset = Dataset.from_pandas(df)

if max_samples > 0 and max_samples < len(dataset):
    dataset = dataset.shuffle(seed=3407).select(range(min(max_samples, len(dataset))))

train_val_idx, test_idx = train_test_split(
    range(len(dataset)),
    test_size=0.2,
    random_state=3407
)

train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.125, # 0.125 * 0.8 = 0.1
    random_state=3407
)

def process_training_example(example):
    text = example['abstract']
    is_ai_generated = example['is_ai_generated']
    prompt = (
        "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
        f"Given Text: {text}\n\n"
        "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        f"Answer: {str(is_ai_generated)}"
    )
    return {"text": prompt}

def process_test_example(example):
    text = example['abstract']
    prompt = (
        "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
        f"Given Text: {text}\n\n"
        "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        "Answer:"
    )
    return prompt

train_examples = [process_training_example(dataset[i]) for i in train_idx]
eval_examples = [process_training_example(dataset[i]) for i in val_idx]
test_examples = [dataset[i] for i in test_idx]

train_dataset = Dataset.from_list(train_examples)
eval_dataset = Dataset.from_list(eval_examples)
test_dataset = Dataset.from_list([
    {'abstract': item['abstract'], 'is_ai_generated': item['is_ai_generated']} for item in test_examples
])

print(f"Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}, Test size: {len(test_dataset)}")

FastLanguageModel.for_inference(model)

batch_size = 16
test_examples_list = list(test_dataset)
total_batches = (len(test_examples_list) + batch_size - 1) // batch_size
all_predictions = []

with torch.no_grad():
    for i in range(0, len(test_examples_list), batch_size):
        if i % (batch_size * 10) == 0:
            print(f"Processing batch {i//batch_size}/{total_batches}")

        batch = test_examples_list[i:i + batch_size]
        prompts = [process_test_example(example) for example in batch]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=8,
            temperature=0.1,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

        input_length = inputs['input_ids'].shape[1]
        responses = tokenizer.batch_decode(
            [output[input_length:] for output in outputs],
            skip_special_tokens=True
        )

        batch_predictions = [parse_prediction(response) for response in responses]
        all_predictions.extend(batch_predictions)

print(f"Total predictions: {len(all_predictions)}")

true_labels = [row['is_ai_generated'] == 'True' for row in test_dataset]

submission_df = pd.DataFrame({
    'ID': range(len(all_predictions)),
    'is_ai_generated_pred': all_predictions,
    'is_ai_generated_true': true_labels
})

submission_df['correctness'] = (submission_df['is_ai_generated_pred'] == submission_df['is_ai_generated_true'])

submission_path = os.path.join(save_dir, 'submission_with_truth_5000.csv')
submission_df.to_csv(submission_path, index=False)
print(f"Submission with truth saved to {submission_path}")

print(submission_df.head())

print("Classification Report:")
print(classification_report(true_labels, all_predictions, target_names=["Human","AI"]))
